__качаем объявления о продаже домов с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'

raw_data_path = 'data/raw/house/avito' # каталог с файлами объявлений

In [6]:
from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

---

In [7]:
place = 'sevastopol'
# place = 'bahchisaray'
# place = 'sudak'
avito_path = place+'/doma_dachi_kottedzhi/prodam'

In [8]:
%%time 

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-28 18:20:01 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-28 18:20:01 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-28 18:20:03 | AdsListParser: downloader init
[INFO    ] 2022-11-28 18:20:03 | AvitoParser: init
[INFO    ] 2022-11-28 18:20:07 | AvitoParser: 45 pages for read
[INFO    ] 2022-11-28 18:20:07 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-28 18:20:10 | AdsListParser: read page 1
[INFO    ] 2022-11-28 18:20:12 | AdsListParser: read page 2
[INFO    ] 2022-11-28 18:20:17 | AdsListParser: read page 3
[INFO    ] 2022-11-28 18:20:20 | AdsListParser: read page 4
[INFO    ] 2022-11-28 18:20:24 | AdsListParser: read page 5
[INFO    ] 2022-11-28 18:20:26 | AdsListParser: read page 6
[INFO    ] 2022-11-28 18:20:30 | AdsListParser: read page 7
[INFO    ] 2022-11-28 18:20:33 | AdsListParser: read page 8
[INFO    ] 2022-11-28 18:20:36 | AdsListParser: read page 9
[INFO    ] 2022-11-28 18:20:39 | AdsListParser

In [9]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

2208
2138


In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
525,2208107939,Объявление «Дом 100 м² на участке 4 сот.» 8 фотографий,6400000,,садоводческое товарищество Надежда,"Предлагается к продаже новый дом в чистовой отделке в живописном месте г. Севастополя. Дом построен с соблюдением строительных норм из качественных материалов. Находится в районе Монастырского шоссе. Рядом кооперативы Надежда, Фрегат, Ямал, Катюша. До моря",11,2022-11-28 18:22:27.676881
1350,2493265029,Объявление «Дом 180 м² на участке 4 сот.» 26 фотографий,18000000,,"Муромская ул., 92",Номер объекта: 74331.\nПродам новый 2-этажный дом. Площадь дома 180м2 Дом расположен в уютном микрорайоне. До центра города добираться 5-7 минут на автомобиле. В 150 метрах от дома остановка общественного транспорта. Назначение земли — Ижс. Электричество 15,28,2022-11-28 18:22:27.676881


In [11]:
df['place'] = place

In [12]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw_{place}.xlsx',index=False)

---

In [13]:
# place = 'sevastopol'
place = 'bahchisaray'
# place = 'sudak'
avito_path = place+'/doma_dachi_kottedzhi/prodam'

In [14]:
df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))
df.sample(2)

[INFO    ] 2022-11-28 18:22:28 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-28 18:22:28 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-28 18:22:31 | AdsListParser: downloader init
[INFO    ] 2022-11-28 18:22:31 | AvitoParser: init
[INFO    ] 2022-11-28 18:22:35 | AvitoParser: 6 pages for read
[INFO    ] 2022-11-28 18:22:35 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-28 18:23:31 | AdsListParser: read page 1
[INFO    ] 2022-11-28 18:23:36 | AdsListParser: read page 2
[INFO    ] 2022-11-28 18:24:39 | AdsListParser: read page 3
[INFO    ] 2022-11-28 18:24:41 | AdsListParser: read page 4
[INFO    ] 2022-11-28 18:24:43 | AdsListParser: read page 5
[INFO    ] 2022-11-28 18:24:45 | AdsListParser: read page 6
[INFO    ] 2022-11-28 18:24:45 | AdsListParser: last page detected
[INFO    ] 2022-11-28 18:24:45 | DownloaderSeleniumFirefox: close virtual browser
260


,avito_id,title,price,obj_name,adr,description,page,ts
187,2623444619,"Объявление «Дача 35 м² на участке 5,1 сот.»",650000,,с. Маловидное,"Продается дача, в Снт Энтузиаст, ул. Ягодная, д.29.",4,2022-11-28 18:24:45.486709
223,2408824766,Объявление «Дом 52 м² на участке 2 сот.» 9 фотографий,5300000,,Речная ул.,"Продам дом, заходи и живи ремонт не требуется, газ, свет, вода.\nВ 5 минутах Хан-дворец и парк Миниатюр.\nСделан ремонт, тёплый пол во всём доме, заменена вся электрика до остановки 2 мин местные достопримечательности в 5-10мин.\n\nВсе вопросы по телефону.\nТор",5,2022-11-28 18:24:45.486709


In [15]:
df['place'] = place
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw_{place}.xlsx',index=False)

----

In [16]:
# place = 'sevastopol'
# place = 'bahchisaray'
place = 'sudak'
avito_path = place+'/doma_dachi_kottedzhi/prodam'

In [17]:
df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))
df.sample(2)

[INFO    ] 2022-11-28 18:24:45 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-28 18:24:45 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-28 18:24:48 | AdsListParser: downloader init
[INFO    ] 2022-11-28 18:24:48 | AvitoParser: init
[INFO    ] 2022-11-28 18:24:53 | AvitoParser: 3 pages for read
[INFO    ] 2022-11-28 18:24:53 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-28 18:24:55 | AdsListParser: read page 1
[INFO    ] 2022-11-28 18:24:57 | AdsListParser: read page 2
[INFO    ] 2022-11-28 18:25:02 | AdsListParser: read page 3
[INFO    ] 2022-11-28 18:25:02 | AdsListParser: last page detected
[INFO    ] 2022-11-28 18:25:02 | DownloaderSeleniumFirefox: close virtual browser
108


,avito_id,title,price,obj_name,adr,description,page,ts
35,2658303608,Объявление «Дом 36 м² на участке 5 сот.»,6000000,,ул. Майора Хвостова,"Продается дом в г. Судак, черта города. Ижс. Документы РФ. Дом в собственности. Участок 5 соток, на стадии оформления. Школа, поликлиниа, больница, магазины, рынок, автостанция в 5-ти минутах хотьбы. До моря 15 минут. Заведена вода и электричество. Газ по",1,2022-11-28 18:25:02.328515
78,2575099970,Объявление «Дом 310 м² на участке 6 сот.» 14 фотографий,18000000,,"с. Миндальное, СНТ Солнечная Долина, Тихий пер., 927","Цена актуальна, пока не продан интересующий меня объект….\n\nПродам дом 310 кв. Метров, с пропиской. На видовом участке 6 соток. С балконов чудесный вид на море и горы. Четыре этажа, отдельный вход на каждый этаж. На каждом этаже 3 комнатная квартира с конди",2,2022-11-28 18:25:02.328515


In [18]:
df['place'] = place
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw_{place}.xlsx',index=False)